# 작사가 만들기

## 1. 데이터 다운로드
```~/aiffel/lyricist/data/lyrics```에 데이터가 있습니다.

## 2. 데이터 읽어오기
* ```glob```를 활용해서 모든 txt파일을 읽어온 후,
* ```raw_corpus```리스트에 문장 단위로 저장!

In [14]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

In [15]:
# library import
import os, re 
import numpy as np
import tensorflow as tf

## 3. 데이터 전처리
* ```preprocess_sentence()``` 함수를 활용해서 데이터를 정제!


### 3-1. 데이터 파악

In [16]:
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you', 'By my side But then I spent so many nights', "Just thinking how you've done me wrong", 'I grew strong', "I learned how to get along And so you're back", 'From outer space', 'I just walked in to find you', 'Here without that look upon your face I should have changed that fucking lock']


* 특별히 처음부터 제거할만한 데이터(문장)는 없는 것 같다.

### 3-2. Tokenise
* 소문자로 바꾸고, 양쪽 공백을 지움 #1

* 특수문자 양쪽에 공백넣기 #2

* 여러개의 공백을 하나의 공백으로 바꿈 #3
  
* a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈 #4
  
* 다시 양쪽 공백을 지움 #5

* 문장 시작에는 ```<start>```, 끝에는 ```<end>```를 추가 #6

In [29]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 한번 사용해보자.
print(preprocess_sentence("This @_is ;;;sample        sentence."))
print(preprocess_sentence("Just thinking how you've done me wrong")) # 라인9
print(preprocess_sentence("Here without that look upon your face I should have changed that fucking lock")) # 라인10

<start> this is sample sentence . <end>
<start> just thinking how you ve done me wrong <end>
<start> here without that look upon your face i should have changed that fucking lock <end>


### 3-3. 단어사전 만들기 (데이터셋 구축)

In [60]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    # 지나치게 긴 문장은 과도한 padding을 갖게 하므로 제거합니다.
    # 지나치게 긴 문장: 토큰의 개수가 15개를 넘어가는 문장
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) >= 30: continue
        
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 100개만 확인
corpus[:100]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i grew strong <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> walk out the door <end>',
 '<start> just turn around <end>',
 '<start> did you think i d crumble ? <end>',
 '<start> i will survive <end>',
 '<start> yeah <end>',
 '<start> i ve got all my life to live <end>',
 '<start> i ve got all my love to give <end>',
 '<start> yeah , yeah <end>',
 '<start> of my broken heart <end>',
 '<start> and i spent oh so many nights <end>',
 '<start> and you see me <end>',
 '<start> still in love with you <end>',
 '<start> for someone whose lovin me <end>',
 '<start> well now go , <end>',
 '<start> weren t you the one <end>',
 '<start> oh not i , <end>',
 '<start> i will survive yeah <end>',
 '<start> i know i ll be alive <end>',
 '<start> i will survive , <end>',
 '<start> i will survive <end>',
 '<start> yeah , yeah <end>',
 '<start> she s putting up her hair <

### 3-4. 데이터를 숫자로 변환
* 텐서플로우 모듈을 활용한 vectorise
* 단어장의 크기는 12000이상

In [66]:
def tokenize(corpus):
    # 1. 단어장 만들기
    # 1-1. Tokenise
    # 12000단어를 기억할 수 있는 tokenizer를 만듦
    # 우리는 이미 문장을 정제했으니 filters가 필요없음
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # 1-2. corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 2. Translation (corpus -> Tensor)
    # 2-1. 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 2-2. 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # **문장 뒤에 패딩을 붙여 길이를 맞춰줍니다. (만약 시퀀스가 짧다면)
    # **문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   76  321 ...    0    0    0]
 [   2    4   60 ...    0    0    0]
 [   2    4 1831 ...    0    0    0]
 ...
 [   2 2050    8 ...    0    0    0]
 [   2    4   21 ...    0    0    0]
 [   2   50   16 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f77b33d17d0>


In [79]:
# 텐서를 소스와 타겟으로 분리
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0]) # 소스 문장
print(tgt_input[0]) # 타겟 문장

[  2  76 321   4  60 636   3   0   0   0   0   0   0   0   0   0   0   0
   0]
[ 76 321   4  60 636   3   0   0   0   0   0   0   0   0   0   0   0   0
   0]


### 3-5. 데이터셋 객체 생성

In [88]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개 + 0:<pad> = 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset # 데이터셋 준비 완료! 전처리 끝

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

### Step 3-6. 데이터셋 분리
* 총 데이터의 20%를 평가데이터셋으로 활용

In [64]:
from sklearn.model_selection import train_test_split

# train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, stratify=None, random_state=34)

In [65]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (54672, 19)
Target Train: (54672, 19)


## 4. 인공지능 만들기
* Embedding Size와 Hidden Size를 조절하며, 10 Epoch안에 ```val_loss``` 값을 2.2 수준으로 줄일 수 있는 모델 설계

In [80]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [81]:

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
266/266 [==============================] - 322s 1s/step - loss: 2.3096
Epoch 2/10
266/266 [==============================] - 305s 1s/step - loss: 1.5219
Epoch 3/10
266/266 [==============================] - 307s 1s/step - loss: 1.3990
Epoch 4/10
266/266 [==============================] - 308s 1s/step - loss: 1.2976
Epoch 5/10
266/266 [==============================] - 308s 1s/step - loss: 1.2057
Epoch 6/10
266/266 [==============================] - 308s 1s/step - loss: 1.1248
Epoch 7/10
266/266 [==============================] - 308s 1s/step - loss: 1.0491
Epoch 8/10
266/266 [==============================] - 307s 1s/step - loss: 0.9848
Epoch 9/10
266/266 [==============================] - 307s 1s/step - loss: 0.9310
Epoch 10/10
266/266 [==============================] - 307s 1s/step - loss: 0.8794


## 5. 평가

In [86]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [87]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

## 6. Comments
* 처음에 에폭수를 100으로 해서 시도했을 때는 loss는 지금보다 높고, i love the way you lie를 출력해 주었다.
* 지금은 loss가 확 낮아지고 I love you가 뜬 것으로 보아, I love 다음에 you가 나오는 경우가 상대적으로 더 많은 것 같다.